In [ ]:
import os
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
# Load API key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# -------------------------------
# Helper: Prompt Builder
# -------------------------------
def build_prompt(task, topic, tone, audience):
    task_prompts = {
        "Brochure": f"Write a compelling marketing brochure about {topic}.",
        "Blog Post": f"Write a blog post on {topic} with engaging storytelling and useful insights.",
        "Product Comparison": f"Write a product comparison summary focusing on {topic}, including pros, cons, and recommendations.",
        "Idea Brainstorm": f"Brainstorm creative ideas or solutions related to {topic}."
    }
    base = task_prompts.get(task, "Write something creative.")
    if tone:
        base += f" Use a {tone} tone."
    if audience:
        base += f" Tailor it for {audience}."
    return base

In [ ]:
# -------------------------------
# Generate with multiple models
# -------------------------------
def generate_stream(task, topic, tone, audience, model):
    if not topic.strip():
        yield "⚠️ Please enter a topic."
        return

    prompt = build_prompt(task, topic, tone, audience)

    stream = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=800,
        stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
# -------------------------------
# Refinement logic
# -------------------------------
def refine_stream(original_text, instruction, model):
    if not original_text.strip():
        yield "⚠️ Please paste the text you want to refine."
        return
    if not instruction.strip():
        yield "⚠️ Please provide a refinement instruction."
        return

    refined_prompt = f"Refine the following text based on this instruction: {instruction}\n\nText:\n{original_text}"

    stream = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a writing assistant."},
            {"role": "user", "content": refined_prompt}
        ],
        max_tokens=800,
        stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result


In [ ]:
# -------------------------------
# Gradio UI
# -------------------------------
with gr.Blocks(title="AI Creative Studio") as demo:
    gr.Markdown("# AI Creative Studio\nGenerate marketing content, blog posts, or creative ideas — streamed in real-time!")

    with gr.Row():
        task = gr.Dropdown(
            ["Brochure", "Blog Post", "Product Comparison", "Idea Brainstorm"],
            label="Task Type",
            value="Brochure"
        )
        topic = gr.Textbox(label="Topic", placeholder="e.g., Electric Cars, AI in Education...")
    with gr.Row():
        tone = gr.Textbox(label="Tone (optional)", placeholder="e.g., professional, casual, humorous...")
        audience = gr.Textbox(label="Target Audience (optional)", placeholder="e.g., investors, students, developers...")

    model = gr.Dropdown(
        ["gpt-4o-mini", "gpt-3.5-turbo", "gpt-4"],
        label="Choose a model",
        value="gpt-4o-mini"
    )

    generate_btn = gr.Button("Generate Content")
    output_md = gr.Markdown(label="Generated Content", show_label=True)

    generate_btn.click(
        fn=generate_stream,
        inputs=[task, topic, tone, audience, model],
        outputs=output_md
    )

    gr.Markdown("---\n## Refine Your Content")

    original_text = gr.Textbox(
        label="Original Content",
        placeholder="Paste content you want to refine...",
        lines=10
    )
    instruction = gr.Textbox(
        label="Refinement Instruction",
        placeholder="e.g., Make it shorter and more persuasive.",
    )
    refine_model = gr.Dropdown(
        ["gpt-4o-mini", "gpt-3.5-turbo", "gpt-4"],
        label="Model for Refinement",
        value="gpt-4o-mini"
    )

    refine_btn = gr.Button("Refine")
    refined_output = gr.Markdown(label="Refined Content", show_label=True)

    refine_btn.click(
        fn=refine_stream,
        inputs=[original_text, instruction, refine_model],
        outputs=refined_output
    )


In [ ]:
# -------------------------------
# Launch the App
# -------------------------------
if __name__ == "__main__":
    demo.launch()